# Meta-Regression
From our Data Extraction sheet, we obtain the values for the mean ages for each study. Using these values, we test to see if there is any significant impact of the values on our results

In [ ]:
pip install PythonMeta

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.2/90.2 kB 2.5 MB/s eta 0:00:00


In [ ]:
import PythonMeta as PMA
import math
import numpy as np
import statsmodels.api as sm
from tabulate import tabulate

ages = {
'Brief': 32.1,
'Engle': 36.45,
'hobfoll': 32.4,
'Knaevelsrud': 28.1,
'Krupnick': 40,
'leslieAmorland':46.45,
'Litz': 39.6,
'MariekevanMeggelen' :39.3,
'Niemeyer':37.8,
'Pang': 43.8,
'Possemato':39,
'Stevens': 34.36}


def showresults(rults):
    text = "%-10s %-6s  %-18s %-10s"%("Study ID","n","ES[95% CI]","Weight(%)\n")
    for i in range(1,len(rults)):
        text += "%-10s %-6d  %-4.2f[%.2f %.2f]   %6.2f\n"%(   # for each study
        rults[i][0],     #study ID
        rults[i][5],     #total num
        rults[i][1],     #effect size
        rults[i][3],     #lower of CI
        rults[i][4],     #higher of CI
        100*(rults[i][2]/rults[0][2])  #weight
        )
    text += "%-10s %-6d  %-4.2f[%.2f %.2f]   %6d\n"%(         # for total effect
        rults[0][0],     #total effect size name
        rults[0][5],     #total N (all studies)
        rults[0][1],     #total effect size
        rults[0][3],     #total lower CI
        rults[0][4],     #total higher CI
        100
        )
    text += "%d studies included (N=%d)\n"%(len(rults)-1,rults[0][5])
    text += "Heterogeneity: Tau\u00b2=%.3f "%(rults[0][12]) if not rults[0][12]==None else "Heterogeneity: "
    text += "Q(Chisquare)=%.2f(p=%s); I\u00b2=%s\n"%(
        rults[0][7],     #Q test value
        rults[0][8],     #p value for Q test
        str(round(rults[0][9],2))+"%")   #I-square value
    text += "Overall effect test: z=%.2f, p=%s\n"%(rults[0][10],rults[0][11])  #z-test value and p-value

    return text

setting = {"datatype": "CATE",
                "models": "random",
                "algorithm": "iv",
                "effect": "RR"}

In [ ]:
d = PMA.Data()  # Load Data class
m = PMA.Meta()  # Load Meta class


d.datatype = setting["datatype"]
studies = d.getdata(d.readfile("/content/Outcome_Stats.txt"))
m.datatype = d.datatype
m.models = setting["models"]
m.algorithm = setting["algorithm"]
m.effect = setting["effect"]
results = m.meta(studies)

#print(showresults(results))
y_studyid = []
for i in range(1,len(results)):
    y_studyid.append([results[i][0], results[i][1]])
x = []          # Age
y = []          # Effect size
for i in range(len(y_studyid)):
    x.append(ages[y_studyid[i][0]])
    y.append(y_studyid[i][1])
x = np.array(x)
y = np.array(y)
x = sm.add_constant(x)
est = sm.OLS(y, x).fit()
print(est.summary())
if est.pvalues[1] > 0.05:
    print("\nThe tested variable is not statistically significant\n")
else:
    print("\nThe tested variable is statistically significant\n")

                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.133
Model:                            OLS   Adj. R-squared:                  0.046
Method:                 Least Squares   F-statistic:                     1.536
Date:                Mon, 02 Oct 2023   Prob (F-statistic):              0.243
Time:                        20:16:00   Log-Likelihood:                -4.8597
No. Observations:                  12   AIC:                             13.72
Df Residuals:                      10   BIC:                             14.69
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.0839      0.881      0.095      0.9

/usr/local/lib/python3.10/dist-packages/scipy/stats/_stats_py.py:1806: UserWarning: kurtosistest only valid for n>=20 ... continuing anyway, n=12
  warnings.warn("kurtosistest only valid for n>=20 ... continuing "


# Eggers Regression
Now we use the Eggers regression to test for Publication Bias in our Systematic Review

In [ ]:
est1 = m.Eggers_test(results)
print("Output for Egger's test:")
allData = [['Intercept', 't-value', 'p-value', 'S.D', 'Confidence Intervals'], est1]
print(tabulate(allData))
if est1[2] > 0.05:
    print("No publication bias!")
else:
    print("There is publication bias.")

Output for Egger's test:
------------------  ------------------  -------------------  ------------------  -------------------------
Intercept           t-value             p-value              S.D                 Confidence Intervals
0.5857183363490212  0.7866448869521221  0.44972989425607424  0.7445778216628388  [-1.07330444  2.24474111]
------------------  ------------------  -------------------  ------------------  -------------------------
No publication bias!
